# Import Modules

In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import os

# Set Hyperparameter

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(77)
if device == 'cuda':
    torch.cuda.manual_seed_all(77)
print(f"uning: {device}")

batch_size = 100
learning_rate = 1e-3
num_epoch = 200

uning: cuda


# Load Data

In [3]:
# def get_mnist(batch_size=batch_size, image_size=28):
#     #download data
#     mnist_train = dset.MNIST(root = 'MNIST_data/',
#                              train=True, # train data로 download
#                              transform=transforms.Compose([
#                                  transforms.ToTensor(),
#                                  transforms.Normalize(mean=(0.1307,), std=(0.3081,))
#                              ]),
#                              target_transform=None,
#                              download=True)

#     mnist_test = dset.MNIST(root = 'MNIST_data/',
#                              train=False, 
#                              transform=transforms.Compose([
#                                  transforms.ToTensor(),
#                                  transforms.Normalize(mean=(0.1307,), std=(0.3081,))
#                              ]),
#                              target_transform=None,
#                              download=True)

#     train_loader = DataLoader(mnist_train,
#                               batch_size=batch_size,
#                               shuffle=True,
#                               drop_last=True,
#                               num_workers=8) # data processing에 할당하는 cpu core수

#     test_loader = DataLoader(mnist_test,
#                              batch_size=batch_size,
#                              shuffle=True,
#                              drop_last=True,
#                              num_workers=8)
    
#     return (train_loader, test_loader)



def get_alphabet(root: str, batch_size: int):
    
    train_path = os.path.join(root, 'train')
    test_path = os.path.join(root, 'test')
    
    alphabet_train1 = ImageFolder(root = train_path,
                                 transform=transforms.Compose([
                                     transforms.ToTensor(),
#                                      transforms.Normalize(mean=(0.1307,), std=(0.3081,)),
                                     transforms.Grayscale(1),
                                     transforms.RandomRotation(5),
                                     transforms.RandomInvert()
                                 ]),
                                 target_transform=None)
    
    alphabet_train2 = ImageFolder(root = train_path,
                                 transform=transforms.Compose([
                                     transforms.ToTensor(),
#                                     transforms.Normalize(mean=(0.1307,), std=(0.3081,)),
                                     transforms.Grayscale(1)
                                 ]),
                                 target_transform=None)
    
    alphabet_train3 = ImageFolder(root = train_path,
                                 transform=transforms.Compose([
                                     transforms.ToTensor(),
#                                     transforms.Normalize(mean=(0.1307,), std=(0.3081,)),
                                     transforms.Grayscale(1),
                                     transforms.CenterCrop(20),
                                     transforms.Resize(28)
                                 ]),
                                 target_transform=None)
    
    
    
    alphabet_test = ImageFolder(root = test_path,
                                 transform=transforms.Compose([
                                     transforms.ToTensor(),
#                                      transforms.Normalize(mean=(0.1307,), std=(0.3081,)),
                                     transforms.Grayscale(1)
                                 ]),
                                 target_transform=None)
    
    train_loader = DataLoader(alphabet_train1,
                              batch_size=batch_size,
                              shuffle=True,
                              drop_last=True,
                              num_workers=8)

    test_loader = DataLoader(alphabet_test,
                             batch_size=batch_size,
                             shuffle=False,
                             drop_last=False,
                             num_workers=8) 
    
    return (train_loader, test_loader)


In [4]:
dset_root = './data'
train_loader, test_loader = get_alphabet(root = dset_root, batch_size = batch_size)
#get_mnist()
#get_alphabet(root = dset_root, batch_size = batch_size)

# Check data

In [5]:
# img, target = train_loader.__getitem__(100)
# img = transforms.ToPILImage()
# print(train_loader.class_to_index)
# print(f"shape: {img.shape}"

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, padding=1)
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(in_features=128*4*4, out_features=1000),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1000, 26)
        )
        
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.size(0), -1)
        out = self.fc_layer(out)
        return out


In [7]:
model = CNN().to(device)

# train1_2_ pretrained

# model = torch.load("./weights/alphabet_45ep_0.9461762309074402.pth").to(device)
loss_func = nn.CrossEntropyLoss().to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-1)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-5)


In [13]:
def accuracy(model, test_loader):
    correct = 0
    total = 0
#     path = "./weights/AdamW_alphabet_47ep_0.0013882338535040617.pth"
#     test_model = torch.load(path)
#     test_model.eval()
    model.eval()
    with torch.no_grad():
        for image, label in test_loader:
            x = image.to(device)
            y_= label.to(device)

            output = test_model.forward(x)
            _, output_index = torch.max(output, 1)

            total += label.size(0)
            correct += (output_index == y_).sum().float()

        return f"{100.0*correct/total}"

In [14]:
from tqdm import tqdm


for i in range(1, num_epoch+1):
#     for _,[image,label] in tqdm(enumerate(train_loader)):
    for _,[image,label] in enumerate(train_loader):
        x = image.to(device)
        y_= label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
    
    accuracy.append(accuracy(model, test_loader))
    if (np.max(accuracy) <= accuracy[i]):
        path = f"./weights/AdamW_alphabet_{i}ep_{accuarcy[i]}.pth"
        torch.save(model, path)
    
    scheduler.step(loss)
    print(f"Epoch: {i}, Loss: {loss.item()}, LR: {scheduler.optimizer.state_dict()['param_groups'][0]['lr']}")
    

TypeError: 'list' object is not callable

In [ ]:
import numpy as np
np.min(losses)

In [ ]:
print(path)